In [77]:
import os
os.chdir('D:\\Test_ML')


In [78]:
%pwd


'D:\\Test_ML'

In [79]:
import os
print(os.listdir())  # List contents of the current directory


['.env', '.git', '.github', '.gitignore', 'app.py', 'artifacts', 'config', 'Dockerfile', 'logs', 'main.py', 'params.yaml', 'README.md', 'requirements.txt', 'research', 'schema.yaml', 'setup.py', 'src', 'template.py', 'venv']


In [80]:
import sys
import os

# Add the 'src' directory to the path

sys.path.append(os.path.abspath("src"))

In [81]:
## src\diseaseprognosis\entity\config_entity.py
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [82]:
## src\diseaseprognosis\config\configuration.py
from src.rottentomatoes.constants import *
from src.rottentomatoes.utils.common import read_yaml, create_directories

In [83]:
## Now we will be creating configuration manager
## src\diseaseprognosis\config\configuration.py
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [84]:
import os
import urllib.request as request
import zipfile
from src.rottentomatoes import logger
from src.rottentomatoes.utils.common import get_size

In [85]:
## src\diseaseprognosis\components\data_ingestion.py
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [92]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-12-16 22:57:10,008: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-16 22:57:10,013: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-16 22:57:10,018: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-12-16 22:57:10,021: INFO: common: created directory at: artifacts]
[2024-12-16 22:57:10,022: INFO: common: created directory at: artifacts/data_ingestion]
[2024-12-16 22:57:12,683: INFO: 1612220431: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 11326744
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "56e58931bd88294a771c14a647adb839a56497eba9dcb9fdd5f5cacbc8a6c581"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 4057:2E92ED:6634F1:80A782:676062B1
Accept-Ranges: bytes
Date: Mon,